In [1]:
clear()

## Discription:

This is my test code for my ETF rebate financial algorithm for a research project I did over the summer of 2024. The idea was to test it to see if my algorithm correctly modeled every daily rebate for one ETF. Then I was to test and see if it would work on two to make sure it could be applied to all ETFs. Once it passed those two tests, I was able to use Bridges 2 supercomputer to apply it to over 700 etfs at the same time. The idea was to be able to clean every data set simulatniously accounting for all possiblites to ensure correct calculations for all of the ETFs. 

------------------

### Test 1:

This algorithm leverages financial modeling techniques to process and analyze ETF, rebate, and federal funds rate (FFR) data. It dynamically imputes missing ticker symbols, ensuring data integrity, then merges the financial datasets based on date and matching identifiers to create a comprehensive view of ETF constituents. By filtering out high-weight positions and excluding non-relevant bond data, the model refines its dataset for more accurate analysis. It employs advanced data cleaning methods, including missing utilization imputation and multiple rebate calculations based on both 100% and actual utilization. The results are aggregated at the ETF and date level, producing actionable insights into daily rebates and their adjustments. The model outputs aggregated and merged datasets in CSV format, enabling deeper financial analysis and decision-making in portfolio management.

In [ ]:
import pandas as pd
import numpy as np

def fill_missing_tickers(etf_data):
    missing_tickers = etf_data[etf_data['Constituent_Ticker'].isna()]
    for index, row in missing_tickers.iterrows():
        matching_rows = etf_data[(etf_data['Constituent_Name'] == row['Constituent_Name']) & (etf_data['Constituent_Ticker'].notna())]
        if not matching_rows.empty:
            etf_data.at[index, 'Constituent_Ticker'] = matching_rows.iloc[0]['Constituent_Ticker']
    return etf_data

def analyze_missing_utilization(etf_file, rebate_data_file, ffr_data_file):
    # Read and process rebate data
    rebate_data = pd.read_csv(rebate_data_file, parse_dates=['DataDate'], low_memory=False)
    rebate_data['BB_TICKER'] = rebate_data['BB_TICKER'].str.upper().str.strip()
    rebate_data['ISIN'] = rebate_data['ISIN'].str.upper().str.strip()
    rebate_data['SEDOL'] = rebate_data['SEDOL'].str.upper().str.strip()
    rebate_data['InstrumentName'] = rebate_data['InstrumentName'].str.upper().str.strip()

    ffr_data = pd.read_csv(ffr_data_file, parse_dates=['DATE'], low_memory=False)
    ffr_data.rename(columns={'DATE': 'DataDate', 'DFF': 'FedFundsRate'}, inplace=True)
    ffr_data['FedFundsRate'] = ffr_data['FedFundsRate'].astype(float) * 100 / 365  # Convert FFR to bps per day

    rebate_data = rebate_data.merge(ffr_data, on='DataDate', how='left')
    
    rebate_data['IndicativeRebate'] = rebate_data['IndicativeRebate'].abs() * 10000 / 360  # Take absolute value and convert to BPS

    etf_data = pd.read_csv(etf_file, parse_dates=['As_Of_Date'], low_memory=False)
    etf_data['Constituent_Ticker'] = etf_data['Constituent_Ticker'].str.upper().str.strip()
    etf_data['ISIN'] = etf_data['ISIN'].str.upper().str.strip()
    etf_data['SEDOL'] = etf_data['SEDOL'].str.upper().str.strip()
    etf_data['Constituent_Name'] = etf_data['Constituent_Name'].str.upper().str.strip()

    etf_data = fill_missing_tickers(etf_data)

    etf_name = etf_file.split('_')[-1].split('.')[0]
    etf_data['ETF'] = etf_name

    # Filter out rows where 'Weight' exceeds 0.25
    etf_data = etf_data[etf_data['Weight'] <= 0.25]

    keys_to_try = [
        ('Constituent_Ticker', 'BB_TICKER'),
        ('Constituent_Name', 'InstrumentName'),
        ('SEDOL', 'SEDOL'),
        ('ISIN', 'ISIN')
    ]

    merged_data = pd.DataFrame()
    for etf_key, rb_key in keys_to_try:
        if etf_data[etf_key].isnull().all() or rebate_data[rb_key].isnull().all():
            continue

        temp_merged = etf_data.merge(rebate_data, left_on=['As_Of_Date', etf_key], right_on=['DataDate', rb_key], how='left', indicator=True)

        valid_rows = temp_merged[(pd.notnull(temp_merged[etf_key])) & (pd.notnull(temp_merged[rb_key]))]

        if not valid_rows.empty:
            if merged_data.empty:
                merged_data = valid_rows.copy()
            else:
                merged_data = pd.concat([merged_data, valid_rows])

    if merged_data.empty:
        print(f"No valid rows after merging for ETF: {etf_name}")
        return

    # Ensure 'InstrumentName' is a string before filtering
    merged_data['InstrumentName'] = merged_data['InstrumentName'].astype(str)

    # Exclude bonds by filtering out rows with '%' in InstrumentName
    merged_data = merged_data[~merged_data['InstrumentName'].str.contains('%')]

    # Drop duplicate rows for the same ETF-stock-day combination, keeping the first row only
    merged_data = merged_data.drop_duplicates(subset=['As_Of_Date', 'Constituent_Ticker'], keep='first')

    # Count missing utilization values
    missing_util_count = merged_data['Utilisation'].isna().sum()
    print(f"ETF: {etf_name}, Missing Utilisation Count: {missing_util_count}")

    # Calculate Daily Rebate assuming 100% Utilisation
    merged_data['Daily_Rebate_100_Util'] = merged_data['Weight'] * merged_data['IndicativeRebate']

    # Calculate Adjusted Daily Rebate assuming 100% Utilisation
    merged_data['Adjusted_Daily_Rebate_100_Util'] = merged_data['Weight'] * (merged_data['IndicativeRebate'] + merged_data['FedFundsRate'])

    # Handle missing Utilisation by imputing with 0
    merged_data['Utilisation'] = merged_data['Utilisation'] / 100

    # Calculate Daily Rebate with actual Utilisation
    merged_data['Daily_Rebate'] = merged_data['Weight'] * merged_data['IndicativeRebate'] * merged_data['Utilisation']

    # Calculate Adjusted Daily Rebate with actual Utilisation
    merged_data['Adjusted_Daily_Rebate'] = merged_data['Weight'] * (merged_data['IndicativeRebate'] + merged_data['FedFundsRate']) * merged_data['Utilisation']

    # Ensure all values are in decimal format
    merged_data['Daily_Rebate_100_Util'] = merged_data['Daily_Rebate_100_Util'].apply(lambda x: f"{x:.10f}")
    merged_data['Adjusted_Daily_Rebate_100_Util'] = merged_data['Adjusted_Daily_Rebate_100_Util'].apply(lambda x: f"{x:.10f}")
    merged_data['Daily_Rebate'] = merged_data['Daily_Rebate'].apply(lambda x: f"{x:.10f}")
    merged_data['Adjusted_Daily_Rebate'] = merged_data['Adjusted_Daily_Rebate'].apply(lambda x: f"{x:.10f}")

    daily_aggregated = merged_data.groupby(['As_Of_Date', 'ETF']).agg({
        'Daily_Rebate': 'sum',
        'Adjusted_Daily_Rebate': 'sum'
    }).reset_index()

    daily_aggregated_100_util = merged_data.groupby(['As_Of_Date', 'ETF']).agg({
        'Daily_Rebate_100_Util': 'sum',
        'Adjusted_Daily_Rebate_100_Util': 'sum'
    }).reset_index()

    final_aggregated = pd.merge(
        daily_aggregated,
        daily_aggregated_100_util,
        on=['As_Of_Date', 'ETF'],
        how='left'
    )

    output_file = f"daily_rebate_{etf_name}.csv"
    final_aggregated.to_csv(output_file, index=False)

    merged_output_file = f"merged_data_{etf_name}.csv"
    merged_data.to_csv(merged_output_file, index=False)
    print(f"Data for ETF {etf_name} saved to {output_file}")
    print(f"Merged data for ETF {etf_name} saved to {merged_output_file}")

# Specify the file paths
etf_file = 'etfg_RALS.csv'
rebate_data_file = 'kkmnljfpapt1m46e.csv'
ffr_data_file = 'DFF.csv'

# Run the analysis
analyze_missing_utilization(etf_file, rebate_data_file, ffr_data_file)


-----------------

### Test 2:

Similarly, the algorithm cacluates the rebates through mergers and conversions. This time however we are able to use multiple ETF data sets.

In [ ]:
import pandas as pd

def fill_missing_tickers(etf_data):
    missing_tickers = etf_data[etf_data['Constituent_Ticker'].isna()]
    for index, row in missing_tickers.iterrows():
        matching_rows = etf_data[(etf_data['Constituent_Name'] == row['Constituent_Name']) & (etf_data['Constituent_Ticker'].notna())]
        if not matching_rows.empty:
            etf_data.at[index, 'Constituent_Ticker'] = matching_rows.iloc[0]['Constituent_Ticker']
    return etf_data

def analyze_missing_utilization(etf_files, rebate_data_file, ffr_data_file):
    # Read and process rebate data
    rebate_data = pd.read_csv(rebate_data_file, parse_dates=['DataDate'], low_memory=False)
    rebate_data['BB_TICKER'] = rebate_data['BB_TICKER'].str.upper().str.strip()
    rebate_data['ISIN'] = rebate_data['ISIN'].str.upper().str.strip()
    rebate_data['SEDOL'] = rebate_data['SEDOL'].str.upper().str.strip()
    rebate_data['InstrumentName'] = rebate_data['InstrumentName'].str.upper().str.strip()

    ffr_data = pd.read_csv(ffr_data_file, parse_dates=['DATE'], low_memory=False)
    ffr_data.rename(columns={'DATE': 'DataDate', 'DFF': 'FedFundsRate'}, inplace=True)
    ffr_data['FedFundsRate'] = ffr_data['FedFundsRate'].astype(float) * 100 / 365  # Converts FFR to bps per day

    rebate_data = rebate_data.merge(ffr_data, on='DataDate', how='left')
    
    rebate_data['IndicativeRebate'] = rebate_data['IndicativeRebate'].abs() * 10000 / 360   #takes absolute value and covnert to BPS  

    for etf_file in etf_files:
        etf_data = pd.read_csv(etf_file, parse_dates=['As_Of_Date'], low_memory=False)
        etf_data['Constituent_Ticker'] = etf_data['Constituent_Ticker'].str.upper().str.strip()
        etf_data['ISIN'] = etf_data['ISIN'].str.upper().str.strip()
        etf_data['SEDOL'] = etf_data['SEDOL'].str.upper().str.strip()
        etf_data['Constituent_Name'] = etf_data['Constituent_Name'].str.upper().str.strip()

        etf_data = fill_missing_tickers(etf_data)
        
        etf_name = etf_file.split('_')[-1].split('.')[0]
        etf_data['ETF'] = etf_name

        keys_to_try = [
            ('Constituent_Ticker', 'BB_TICKER'),
            ('Constituent_Name', 'InstrumentName'),
            ('SEDOL', 'SEDOL'),
            ('ISIN', 'ISIN')
        ]

        merged_data = pd.DataFrame()
        for etf_key, rb_key in keys_to_try:
            if etf_data[etf_key].isnull().all() or rebate_data[rb_key].isnull().all():
                continue  
            
            temp_merged = etf_data.merge(rebate_data, left_on=['As_Of_Date', etf_key], right_on=['DataDate', rb_key], how='left', indicator=True)
            
            valid_rows = temp_merged[(pd.notnull(temp_merged[etf_key])) & (pd.notnull(temp_merged[rb_key]))]
            
            if not valid_rows.empty:
                if merged_data.empty:
                    merged_data = valid_rows.copy()
                else:
                    merged_data = pd.concat([merged_data, valid_rows])

        if merged_data.empty:
            continue

        # Ensure 'InstrumentName' is a string before filtering
        merged_data['InstrumentName'] = merged_data['InstrumentName'].astype(str)

        # Exclude bonds by filtering out rows with '%' in InstrumentName
        merged_data = merged_data[~merged_data['InstrumentName'].str.contains('%')]

        # Drop duplicate rows for the same ETF-stock-day combination, keeping the first row only
        merged_data = merged_data.drop_duplicates(subset=['As_Of_Date', 'Constituent_Ticker'], keep='first')

        # Count missing utilization values
        missing_util_count = merged_data['Utilisation'].isna().sum()
        print(f"ETF: {etf_name}, Missing Utilisation Count: {missing_util_count}")

        # Calculate Daily Rebate assuming 100% Utilisation
        merged_data['Daily_Rebate_100_Util'] = merged_data['Weight'] * merged_data['IndicativeRebate']

        # Calculate Adjusted Daily Rebate assuming 100% Utilisation
        merged_data['Adjusted_Daily_Rebate_100_Util'] = merged_data['Weight'] * (merged_data['IndicativeRebate'] + merged_data['FedFundsRate'])

        # Handle missing Utilisation by imputing with 0
        merged_data['Utilisation'] = merged_data['Utilisation'] / 100

        # Calculate Daily Rebate with actual Utilisation
        merged_data['Daily_Rebate'] = merged_data['Weight'] * merged_data['IndicativeRebate'] * merged_data['Utilisation']

        # Calculate Adjusted Daily Rebate with actual Utilisation
        merged_data['Adjusted_Daily_Rebate'] = merged_data['Weight'] * (merged_data['IndicativeRebate'] + merged_data['FedFundsRate']) * merged_data['Utilisation']

        daily_aggregated = merged_data.groupby(['As_Of_Date', 'ETF']).agg({
            'Daily_Rebate': 'sum', 
            'Adjusted_Daily_Rebate': 'sum'
        }).reset_index()

        daily_aggregated_100_util = merged_data.groupby(['As_Of_Date', 'ETF']).agg({
            'Daily_Rebate_100_Util': 'sum', 
            'Adjusted_Daily_Rebate_100_Util': 'sum'
        }).reset_index()

        final_aggregated = pd.merge(
            daily_aggregated, 
            daily_aggregated_100_util, 
            on=['As_Of_Date', 'ETF'], 
            how='left'
        )

        output_file = f"daily_rebate_{etf_name}.csv"
        final_aggregated.to_csv(output_file, index=False)

        merged_output_file = f"merged_data_{etf_name}.csv"
        merged_data.to_csv(merged_output_file, index=False)
        print(f"Data for ETF {etf_name} saved to {output_file}")
        print(f"Merged data for ETF {etf_name} saved to {merged_output_file}")

etf_files = ['etfg_SPY.csv', 'etfg_VNQ.csv']
rebate_data_file = 'kkmnljfpapt1m46e.csv'
ffr_data_file = 'DFF.csv'

analyze_missing_utilization(etf_files, rebate_data_file, ffr_data_file)


-------------